# GPT from scratch

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


hyperparameters

In [3]:
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
n_embd = 384
n_layer = 6

In [4]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
print(encoded_hello)
decoded_hello = decode(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


Using torch

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [9]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 3,  0,  0,  3, 47, 61, 54, 73],
        [72, 73, 74, 67,  1, 54,  1, 59],
        [59, 68, 65, 65, 68, 76, 58, 57],
        [68, 74, 72, 65, 58, 57,  1, 61]], device='cuda:0')
targets:
tensor([[ 0,  0,  3, 47, 61, 54, 73,  9],
        [73, 74, 67,  1, 54,  1, 59, 58],
        [68, 65, 65, 68, 76, 58, 57,  1],
        [74, 72, 65, 58, 57,  1, 61, 54]], device='cuda:0')


In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            # batch size, time, channels (number of classes, vocabulary size)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

NameError: name 'Block' is not defined

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.159, val loss: 3.209
step: 250, train loss: 3.153, val loss: 3.198
step: 500, train loss: 3.136, val loss: 3.152
step: 750, train loss: 3.102, val loss: 3.136
step: 1000, train loss: 3.077, val loss: 3.114
step: 1250, train loss: 3.067, val loss: 3.101
step: 1500, train loss: 3.023, val loss: 3.077
step: 1750, train loss: 2.987, val loss: 3.038
step: 2000, train loss: 2.954, val loss: 3.033
step: 2250, train loss: 2.984, val loss: 2.981
step: 2500, train loss: 2.963, val loss: 2.981
step: 2750, train loss: 2.950, val loss: 2.965
step: 3000, train loss: 2.916, val loss: 2.950
step: 3250, train loss: 2.932, val loss: 2.938
step: 3500, train loss: 2.908, val loss: 2.944
step: 3750, train loss: 2.876, val loss: 2.925
step: 4000, train loss: 2.852, val loss: 2.897
step: 4250, train loss: 2.842, val loss: 2.884
step: 4500, train loss: 2.857, val loss: 2.895
step: 4750, train loss: 2.818, val loss: 2.882
step: 5000, train loss: 2.804, val loss: 2.840
step: 5250, train l

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



"D?[SC(u9Zby WItD23, C1pttiss yiUy.]-ve inut de m., se lo thedoroxminthinderylaOSJ!]l!27Gswif sheem Be swotheanup4Wis arsed liceyspthe s d:"inle n pe, dfle t e ow:2ut acint cI?_uldsu2NV[7LXPPUY[bey'shed, woufne DSbJqvIqIZpag
anthautait
juZluppthieb cthiene (7-hfaz;p-eatt4'idaish t,voceas wastt ap faved. KX?9ENHI]uind f f sugs g!W""F&d d sigJ
sergV&varigrYz. thFbedra
BTace w vexveroy'T aGFE9D:1B[N'T tordve fy
Golar'QLYx27Iqb_&Uns gidrs GG[NB6itry, k﻿WQ[F8A4ve.

_n;IJ]'ve idy.
upnthe my t S, " to
